<a href="https://colab.research.google.com/github/hsn-baqshi/hsn-baqshi/blob/main/AI_agent_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import statistics
from collections import deque

class PuzzleState:
    def __init__(self, board, n):
        self.board = board
        self.n = n
        self.cost = 0
        self.prev_state = None
        self.action = None

    def __eq__(self, other):
        return self.board == other.board

    def __hash__(self):
        return hash(tuple(self.board))

def bfs_search(initial_state):
    n = initial_state.n
    goal_state = PuzzleState(get_goal_state(n), n)

    visited = set()
    queue = deque()
    queue.append(initial_state)

    max_states_concurrently_stored = 0
    solution_depth = 0

    while queue:
        current_state = queue.popleft()

        if current_state == goal_state:
            solution_depth = current_state.cost
            return current_state, max_states_concurrently_stored, solution_depth

        visited.add(current_state)

        for neighbor in get_neighbors(current_state):
            if neighbor not in visited and neighbor not in queue:
                queue.append(neighbor)

            max_states_concurrently_stored = max(max_states_concurrently_stored, len(queue) + len(visited))

    return None, max_states_concurrently_stored, solution_depth

def dfs_search(initial_state):
    n = initial_state.n
    goal_state = PuzzleState(get_goal_state(n), n)

    visited = set()
    stack = [initial_state]

    max_states_concurrently_stored = 0
    solution_depth = 0

    while stack:
        current_state = stack.pop()

        if current_state == goal_state:
            solution_depth = current_state.cost
            return current_state, max_states_concurrently_stored, solution_depth

        visited.add(current_state)

        for neighbor in get_neighbors(current_state):
            if neighbor not in visited and neighbor not in stack:
                stack.append(neighbor)

            max_states_concurrently_stored = max(max_states_concurrently_stored, len(stack) + len(visited))

    return None, max_states_concurrently_stored, solution_depth

def dfs_with_revisit_check(initial_state):
    n = initial_state.n
    goal_state = PuzzleState(get_goal_state(n), n)

    visited = set()
    stack = [initial_state]

    max_states_concurrently_stored = 0
    solution_depth = 0

    while stack:
        current_state = stack.pop()

        if current_state == goal_state:
            solution_depth = current_state.cost
            return current_state, max_states_concurrently_stored, solution_depth

        visited.add(current_state)

        for neighbor in get_neighbors(current_state):
            if neighbor not in visited and neighbor not in stack:
                stack.append(neighbor)

        max_states_concurrently_stored = max(max_states_concurrently_stored, len(stack) + len(visited))

    return None, max_states_concurrently_stored, solution_depth

def print_board(board, n):
    for i in range(n):
        for j in range(n):
            print(board[i * n + j], end="\t")
        print()

def get_goal_state(n):
    return list(range(1, n * n)) + [0]

def get_neighbors(state):
    neighbors = []
    n = state.n
    blank_idx = state.board.index(0)

    moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    for move in moves:
        new_blank_idx = blank_idx + move[0] + move[1] * n

        if 0 <= new_blank_idx < n * n:
            new_board = state.board[:]
            new_board[blank_idx], new_board[new_blank_idx] = new_board[new_blank_idx], new_board[blank_idx]
            new_state = PuzzleState(new_board, n)
            new_state.cost = state.cost + 1
            new_state.prev_state = state
            new_state.action = move
            neighbors.append(new_state)

    return neighbors

def get_solution_sequence(state):
    sequence = []
    while state.prev_state is not None:
        sequence.insert(0, state.action)
        state = state.prev_state

    return sequence

def main():
    num_instances = int(input("Enter the number of instances to solve: "))

    algorithm = input("Enter the algorithm to use for finding the solution (BFS / DFS / DFS with revisit check): ")

    max_states_data = {}
    depth_data = {}

    for i in range(num_instances):
        n = int(input(f"Enter the size of the puzzle board for instance {i+1} (n): "))
        if n < 3:
            print("Puzzle size should be at least 3x3.")
            return

        puzzle = []
        print(f"Enter the initial state for instance {i+1} (0 represents the blank space):")
        for _ in range(n):
            row = list(map(int, input().split()))
            if len(row) != n:
                print(f"Each row should have {n} elements.")
                return
            puzzle.extend(row)

        if sorted(puzzle) != list(range(n * n)):
            print("Invalid initial state. It should contain all numbers from 0 to n^2 - 1.")
            return

        initial_state = PuzzleState(puzzle, n)

        if algorithm == "BFS":
            result_state, max_states_stored, solution_depth = bfs_search(initial_state)
        elif algorithm == "DFS":
            result_state, max_states_stored, solution_depth = dfs_search(initial_state)
        elif algorithm == "DFS with revisit check":
            result_state, max_states_stored, solution_depth = dfs_with_revisit_check(initial_state)
        else:
            print("Invalid algorithm choice. Please choose from 'BFS', 'DFS', or 'DFS with revisit check'.")
            return

        if result_state:
            print(f"Solution found for instance {i+1}! Steps to reach the goal state:")
            print_board(result_state.board, n)
            print(f"Solution depth: {solution_depth}")
            print(f"Maximum states concurrently stored: {max_states_stored}")
            solution_sequence = get_solution_sequence(result_state)
            action_sequence = []
            for action in solution_sequence:
                if action == (0, 1):
                    action_sequence.append("D")
                elif action == (1, 0):
                    action_sequence.append("R")
                elif action == (0, -1):
                    action_sequence.append("D")
                elif action == (-1, 0):
                    action_sequence.append("L")
            print(f"Solution sequence of actions: {' '.join(action_sequence)}")

            if n in max_states_data:
                max_states_data[n].append(max_states_stored)
            else:
                max_states_data[n] = [max_states_stored]

            if n in depth_data:
                depth_data[n].append(solution_depth)
            else:
                depth_data[n] = [solution_depth]
        else:
            print("No solution found.")

    print("\nDescriptive Statistics for Maximum States Concurrently Stored (for Each Solution Instance):")
    for n, max_states in max_states_data.items():
        print(f"For n={n}:")
        print(f"Minimum maximum states concurrently stored: {min(max_states)}")
        print(f"Maximum maximum states concurrently stored: {max(max_states)}")
        print(f"Average maximum states concurrently stored: {statistics.mean(max_states)}")
        print()

    print("\nDescriptive Statistics for Solution Depth (for Each Solution Instance):")
    for n, depths in depth_data.items():
        print(f"For n={n}:")
        print(f"Minimum depth: {min(depths)}")
        print(f"Maximum depth: {max(depths)}")
        print(f"Average depth: {statistics.mean(depths)}")
        print()

if __name__ == "__main__":
    main()


